In [139]:
from constraint import *
import numpy as np
import json

# 简单的问题定义

* 时间段：假设一天有4个TimeSlot
* 教室：2个
* 老师：3个
* 学生：3个
* 课程：6节
* __目标：将6节课分到8个时空里(4个ts*2个教室）__

课程和学生&老师的关系，其中 abc为老师， xyz为学生

* c1: {a, x}
* c2: {b, z}
* c3: {c, x, y}
* c4: {b, x, z}
* c5: {b, y}
* c6: {c, z}


In [150]:
# 这个Json文件格式和上面的list一样。
with open("output.json","r",encoding='utf-8') as f:
    data = json.load(f)
print(data)


[{'33': ['22', '000001', '000002', '000007', '000010', '000020']}, {'20': ['11', '000003', '000004', '000009']}, {'26': ['18', '000003', '000004']}, {'29': ['5', '000003', '000004']}, {'8': ['6', '000005']}, {'25': ['4', '000005', '000009', '000012']}, {'21': ['16', '000006', '000011', '000015', '000025']}, {'24': ['20', '000006', '000011', '000015', '000025']}, {'27': ['15', '000006', '000015', '000025']}, {'37': ['4', '000006', '000011', '000015', '000025']}, {'34': ['14', '000008']}, {'23': ['2', '000009']}, {'18': ['14', '000013', '000014', '000018']}, {'30': ['17', '000016', '000017', '000019']}, {'31': ['19', '000016', '000017', '000019']}, {'35': ['21', '000021', '000022', '000023', '000024']}]


In [151]:
teachers = []
students = []
for c in data :
    for key in c.keys() :
        teachers.append(c[key][0])
        students.extend(c[key][1:])
teachers = list(set(teachers))
students = list(set(students))

teachers.sort()
students.sort()

print(teachers, students)

['11', '14', '15', '16', '17', '18', '19', '2', '20', '21', '22', '4', '5', '6'] ['000001', '000002', '000003', '000004', '000005', '000006', '000007', '000008', '000009', '000010', '000011', '000012', '000013', '000014', '000015', '000016', '000017', '000018', '000019', '000020', '000021', '000022', '000023', '000024', '000025']


In [152]:
# 做一个index map方便使用，为了variable下标方便使用
ind_map_t, ind_map_s = {}, {}

for t, i in zip(teachers, range(len(teachers))):
    ind_map_t[t] = i
    
for s, i in zip(students, range(len(teachers), len(teachers)+len(students))) :
    ind_map_s[s] = i
    

In [153]:
print(ind_map_t, ind_map_s)

{'11': 0, '14': 1, '15': 2, '16': 3, '17': 4, '18': 5, '19': 6, '2': 7, '20': 8, '21': 9, '22': 10, '4': 11, '5': 12, '6': 13} {'000001': 14, '000002': 15, '000003': 16, '000004': 17, '000005': 18, '000006': 19, '000007': 20, '000008': 21, '000009': 22, '000010': 23, '000011': 24, '000012': 25, '000013': 26, '000014': 27, '000015': 28, '000016': 29, '000017': 30, '000018': 31, '000019': 32, '000020': 33, '000021': 34, '000022': 35, '000023': 36, '000024': 37, '000025': 38}


In [154]:
# 通过数组表示课程、学生、老师的关系 
# 一行代表一节课，6个元素为 a b c x y z, 0、1代表是否出席
# 根据输入拼凑成一个r_c的模式
r_c = np.zeros([len(data), len(teachers)+len(students)])

In [155]:
i = 0
for c in data :
    for key in c.keys() :
        t, s   = c[key][0], c[key][1:]
        print(i, t, s)
        r_c[i, ind_map_t[t]] = 1
        for s0 in s :
            r_c[i, ind_map_s[s0]] = 1
        i += 1
r_c

0 22 ['000001', '000002', '000007', '000010', '000020']
1 11 ['000003', '000004', '000009']
2 18 ['000003', '000004']
3 5 ['000003', '000004']
4 6 ['000005']
5 4 ['000005', '000009', '000012']
6 16 ['000006', '000011', '000015', '000025']
7 20 ['000006', '000011', '000015', '000025']
8 15 ['000006', '000015', '000025']
9 4 ['000006', '000011', '000015', '000025']
10 14 ['000008']
11 2 ['000009']
12 14 ['000013', '000014', '000018']
13 17 ['000016', '000017', '000019']
14 19 ['000016', '000017', '000019']
15 21 ['000021', '000022', '000023', '000024']


array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1.,
        0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
        1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 0., 1., 0.,

In [156]:
# 
p = Problem()

In [157]:
#添加决策变量 1~16代表16个可安排的时间（4个时间*4个教室）
#值域0~15， 其中0~15表示课程1~16
p.addVariables(range(1,16),range(0,len(data)))
p._variables

{1: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 2: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 3: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 4: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 5: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 6: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 7: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 8: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 9: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 10: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 11: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 12: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 13: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 14: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
 15: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]}

In [158]:
#添加约束条件：每个时间段只能被分配一个课程（分到6\7为不安排课程）
p.addConstraint(AllDifferentConstraint())

In [132]:
#每个时间里，两个空间中的参与者不重复
def time_constraint(a,b):
    ##############################################
    ##我的课程数现在大于8，这里应该填8吗？
    ##结果应该是无解，但执行一下貌似内存消耗严重
    ##############################################
    if a >=4 or b>= 4: #如果被分到6/7，则不安排课程，不检查约束
        return True
    else:
        return np.all((r_c[a]+r_c[b])<=1) #相邻的时间里参与者不重复（array两行相加所有元素都<=1)

# 对两个相邻的时空（即同一个时间不同教室添加约束）
for i in range(4):
    p.addConstraint(time_constraint,(2*i+1, 2*i+2))
    

In [ ]:
sol = p.getSolutions()
print(sol)

In [138]:
# 简单可视化下结果
cls_map={0:"a,x", 1:'b,z', 2:'c,x,y', 3:'b,x,z', 4:'b,y', 5:'c,z'}

def single_class(i):
    if i>=6:
        return 'o'
    else:
        return (i+1 ,cls_map[i])

ret = sol # {1: 7, 2: 4, 3: 5, 4: 0, 5: 6, 6: 3, 7: 1, 8: 2}

for r in ret :
    for i in range(4):
        a,b = 2*i+1,2*i+2
        print(single_class(r[a]),'|||', single_class(r[b]))
    print("==================================================")

In [30]:
ttt1 = np.array([1,2,3,4,5])
ttt2 = np.array([0,3,4,5,0])

np.any(ttt1<ttt2)

True

In [61]:
X = {"a": [1,2,3], "b": [2,3,4]}
c=[]
for k in X.keys() :
    c.append([k]+X[k])
#     c.extend(str(X[k]))

print(c)

# courses = [ [key].extend(str(X[key])) for key in X.keys() ]
# print(courses)

[['a', 1, 2, 3], ['b', 2, 3, 4]]
